# Gestión de Datos: Práctica 2
## Carlos Ramos Mateos y Adrián Rubio Pintado

Podemos suponer que los datos se recogen a las 3:00 AM cada día. Para obtener un resultado actualizado, solo hay que re-ejecutar este notebook(o transferir el código a un archivo .py).

# Carga de Datos

Definimos una lista de valores para que nuestro lector los considere como missing values o valores nulos cuando los lea.

In [21]:
import pandas as pd
import numpy as np
import sys

missing_values = ["n\a", "na", "--", "NaN", 'None', '']

df_movies = pd.read_csv("movies.csv", na_values = missing_values)

print("Tamaño del dataset: ", df_movies.size)
print("Dimension del dataset: ", df_movies.shape)

Tamaño del dataset:  187269
Dimension del dataset:  (62423, 3)


In [22]:
df_movies.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [23]:
df_ratings = pd.read_csv("ratings.csv")
df_miss = pd.read_csv("ratings.csv", na_values = missing_values)

print("Tamaño del dataset: ", df_ratings.size)
print("Dimension del dataset: ", df_ratings.shape)

Tamaño del dataset:  100000380
Dimension del dataset:  (25000095, 4)


In [24]:
df_ratings.head(10)

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
5,1,1088,4.0,1147868495
6,1,1175,3.5,1147868826
7,1,1217,3.5,1147878326
8,1,1237,5.0,1147868839
9,1,1250,4.0,1147868414


## Tratamiento y limpieza de los datos

Vamos a ver si tenemos missing values mediante isnull() de pandas, que detecta aquellos valores que son NAN o None(incluida la lista de valores que le hemos pasado que considere como missing values)

In [25]:
df_movies.isnull().any()

movieId    False
title      False
genres     False
dtype: bool

In [26]:
df_ratings.isnull().any()

userId       False
movieId      False
rating        True
timestamp    False
dtype: bool

En el dataset de ratings, vemos como la columna rating si tiene valores nulos, vamos a contabilizarlos.

In [27]:
df_ratings.isnull().sum()

userId            0
movieId           0
rating       495115
timestamp         0
dtype: int64

In [36]:
len(df_ratings)

25000095

Sobre el total de 25000095 valoraciones, tenemos que 495115 no tienen un valor para la puntuación númerica en la escala [1,5]. Es decir, tenemos un

In [28]:
495115/25000095 

0.019804524742805977

**0.0198% de valores nulos**

Dado que es un porcentaje bastante bajo, para esta situación consideramos eliminar dichos valores nulos. Dada la naturaleza de la columna y del dataset. Lo más probable es que estos valores sean nulos debido a que la gente escribe comentarios sobre una película, pero no le ha dado un rating. Dado que no sabemos la distribución de comentarios escritos respecto al rating, es decir, si la gente escribe más comentarios cuando no le ha gustado una película porque está enfadada o todo lo contrario, tiende a escribir comentarios cuando le ha encantado la película, tal vez remplazar dichos valores por la media de los ratings totales introduzca cierto sesgo.

Dado que el porcentaje de datos no llega al 2%, podemos permitirnos eliminar dichos registros.

In [74]:
df_ratings_1 = df_ratings.dropna()

In [75]:
len(df_ratings_1)

24504980

Seguimos con la limpieza de nuestro dataset. Vamos a ver los ratings que no entran dentro del rango de la definición. O lo que es lo mismo, detectar outliers.

In [76]:
len(df_ratings_1[df_ratings_1['rating'] > 5])

3062

In [122]:
3062/len(df_ratings_1)

0.00012495418788531197

In [77]:
len(df_ratings_1[df_ratings_1['rating'] < 0])

2230

Obtenemos unos 5000 registros fuera de rango, dado que las puntuaciones en las películas siempre se dan en escala 0-5, y que son un número muy pequeño, que creemos que no cambian la distribución de los ratings, si no que la mantiene, vamos a hacer rescalado de los datos por su valor en el rating aceptado mas cercano(0.5 y 5)

In [124]:
3062/len(df_ratings_1) + 2230/len(df_ratings_1)

0.00021595609480374624

Es decir, tenemos un porcentaje de outliers de **0.0002**

In [107]:
df_ratings_2 = df_ratings_1.copy()
df_ratings_2['rating'].values[df_ratings_2['rating'] > 5] = 5

In [105]:
len(df_ratings_2[df_ratings_2['rating'] > 5])

0

In [109]:
df_ratings_2['rating'].values[df_ratings_2['rating'] < 0] = 0.5

In [110]:
len(df_ratings_2[df_ratings_2['rating'] <0])

0

Dado que  la consulta consiste en ordenar en función de los valores de la media de los ratings, no es necesaria una **normalización** de los datos. **Tampoco es necesario el procesado de las diferentes columnas, ya que no las vamos a usar.**


Tras las tranformaciones hechas, **el porcentaje de outliers y missing values es del 0%**

# Extracción de los generos de la BD

In [111]:
col_genres = df_movies.genres.tolist()

In [112]:
genres = set()
for col in col_genres:
    words = col.split('|')
    genres.update(words)

aux = list(genres)
aux.sort()

In [113]:
genres

{'(no genres listed)',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Espionage Action',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Psychological Thriller',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western'}

In [114]:
no_genres = df_movies.loc[df_movies['genres'].str.contains('(no genres listed)', case=False)]
action = df_movies.loc[df_movies['genres'].str.contains("Action", case=False)]
aventures = df_movies.loc[df_movies['genres'].str.contains("Adventure", case=False)]
animation = df_movies.loc[df_movies['genres'].str.contains("Animation", case=False)]
children = df_movies.loc[df_movies['genres'].str.contains("Children", case=False)]
comedy = df_movies.loc[df_movies['genres'].str.contains("Comedy", case=False)]
crime = df_movies.loc[df_movies['genres'].str.contains("Crime", case=False)]
documentary = df_movies.loc[df_movies['genres'].str.contains("Documentary", case=False)]
drama = df_movies.loc[df_movies['genres'].str.contains("Drama", case=False)]
espionage = df_movies.loc[df_movies['genres'].str.contains("Espionage Action", case=False)]
fantasy = df_movies.loc[df_movies['genres'].str.contains("Fantasy", case=False)]
noir = df_movies.loc[df_movies['genres'].str.contains("Film-Noir", case=False)]
horror = df_movies.loc[df_movies['genres'].str.contains("Horror", case=False)]
imax = df_movies.loc[df_movies['genres'].str.contains("IMAX", case=False)]
musical = df_movies.loc[df_movies['genres'].str.contains("Musical", case=False)]
mystery = df_movies.loc[df_movies['genres'].str.contains("Mystery", case=False)]
psycho = df_movies.loc[df_movies['genres'].str.contains("Psychological Thriller", case=False)]
romance = df_movies.loc[df_movies['genres'].str.contains("Romance", case=False)]
sci_fi = df_movies.loc[df_movies['genres'].str.contains("Sci-Fi", case=False)]
thriller = df_movies.loc[df_movies['genres'].str.contains("Thriller", case=False)]
war = df_movies.loc[df_movies['genres'].str.contains("War", case=False)]
western = df_movies.loc[df_movies['genres'].str.contains("Western", case=False)]

C:\ProgramData\Miniconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


# Consulta

In [118]:
ratings1_nogen = df_ratings_2.loc[df_ratings_2['movieId'].isin(no_genres.movieId.tolist())]
ratings1_action = df_ratings_2.loc[df_ratings_2['movieId'].isin(action.movieId.tolist())]
ratings1_aventures = df_ratings_2.loc[df_ratings_2['movieId'].isin(aventures.movieId.tolist())]
ratings1_animation = df_ratings_2.loc[df_ratings_2['movieId'].isin(animation.movieId.tolist())]
ratings1_children = df_ratings_2.loc[df_ratings_2['movieId'].isin(children.movieId.tolist())]
ratings1_comedy = df_ratings_2.loc[df_ratings_2['movieId'].isin(comedy.movieId.tolist())]
ratings1_crime = df_ratings_2.loc[df_ratings_2['movieId'].isin(crime.movieId.tolist())]
ratings1_documentary = df_ratings_2.loc[df_ratings_2['movieId'].isin(documentary.movieId.tolist())]
ratings1_drama = df_ratings_2.loc[df_ratings_2['movieId'].isin(drama.movieId.tolist())]
ratings1_espionage = df_ratings_2.loc[df_ratings_2['movieId'].isin(espionage.movieId.tolist())]
ratings1_fantasy = df_ratings_2.loc[df_ratings_2['movieId'].isin(fantasy.movieId.tolist())]
ratings1_noir = df_ratings_2.loc[df_ratings_2['movieId'].isin(noir.movieId.tolist())]
ratings1_horror = df_ratings_2.loc[df_ratings_2['movieId'].isin(horror.movieId.tolist())]
ratings1_imax = df_ratings_2.loc[df_ratings_2['movieId'].isin(imax.movieId.tolist())]
ratings1_musical = df_ratings_2.loc[df_ratings_2['movieId'].isin(musical.movieId.tolist())]
ratings1_mystery = df_ratings_2.loc[df_ratings_2['movieId'].isin(mystery.movieId.tolist())]
ratings1_psycho = df_ratings_2.loc[df_ratings_2['movieId'].isin(thriller.movieId.tolist())]
ratings1_romance = df_ratings_2.loc[df_ratings_2['movieId'].isin(romance.movieId.tolist())]
ratings1_sci_fi = df_ratings_2.loc[df_ratings_2['movieId'].isin(sci_fi.movieId.tolist())]
ratings1_thriller = df_ratings_2.loc[df_ratings_2['movieId'].isin(thriller.movieId.tolist())]
ratings1_war = df_ratings_2.loc[df_ratings_2['movieId'].isin(war.movieId.tolist())]
ratings1_western = df_ratings_2.loc[df_ratings_2['movieId'].isin(western.movieId.tolist())]

In [119]:
ratings1 = list()
ratings1.append(ratings1_action['rating'].mean())
ratings1.append(ratings1_aventures['rating'].mean())
ratings1.append(ratings1_animation ['rating'].mean())
ratings1.append(ratings1_children ['rating'].mean())
ratings1.append(ratings1_comedy['rating'].mean())
ratings1.append(ratings1_crime['rating'].mean())
ratings1.append(ratings1_documentary['rating'].mean())
ratings1.append(ratings1_drama['rating'].mean())
ratings1.append(ratings1_espionage['rating'].mean())
ratings1.append(ratings1_fantasy['rating'].mean())
ratings1.append(ratings1_noir['rating'].mean())
ratings1.append(ratings1_horror['rating'].mean())
ratings1.append(ratings1_imax['rating'].mean())
ratings1.append(ratings1_musical ['rating'].mean())
ratings1.append(ratings1_mystery ['rating'].mean())
ratings1.append(ratings1_psycho['rating'].mean())
ratings1.append(ratings1_romance ['rating'].mean())
ratings1.append(ratings1_sci_fi['rating'].mean())
ratings1.append(ratings1_thriller['rating'].mean())
ratings1.append(ratings1_war['rating'].mean())
ratings1.append(ratings1_western['rating'].mean())

table_rat1 = pd.DataFrame(list(zip(list(genres),ratings1)), columns=['genre', 'rating'])

In [121]:
table_rat1.sort_values(by=['rating'], ascending=False).head(10)

,genre,rating
10,Documentary,3.925239
19,Western,3.790905
6,Action,3.705085
5,Horror,3.684708
7,Psychological Thriller,3.676786
14,Espionage Action,3.669913
2,Thriller,3.614761
12,(no genres listed),3.603185
20,IMAX,3.584755
13,Romance,3.554235


Este es el listado de géneros por rating(top 10). Tal vez el género 'sin genero listado' deberíamos omitirlo, pero dado su importancia en el dataset, decicimos mantenerlo. Esto es debido a que hay un número considerablemente alto de peliculas  no catalogadas, que no debemos ignorar, para ser conscientes de las carencia tan importante que supone dicha base de datos en  nuestra consulta.

# Exportación de la consulta al fichero

Dado que ya hemos porgramado la consulta, el modo más eficiente de consultarla, valga la redundancia, es ver únicamente los resultados de la consulta. Generamos un fichero 'pelis_procesadas.csv' con el resutlado de la misma.

In [132]:
table_rat1.to_csv('pelis_procesadas.csv',index=False)

El formato del fichero de salida es 'género' y 'rating medio del género'. Ordenados por el rating medio.

# ---Método Alternativo: Remplazamiento de los outliers

Para ver como afecta nuestra suposición de que los outliers se tenían que sustituir por los valores más cercanos(mínimo o máximo de los ratings), vamos a ver cómo cambiar el ranking de la consulta si cambiamos el método de sustitución por la media de todas las puntuaciones totales.

In [133]:
df_ratings_3 = df_ratings.dropna()


median = df_ratings_3['rating'].between(0.5, 5).median()
df_ratings_3.loc[df_ratings_3['rating'] > 5, 'rating'] = median
df_ratings_3.loc[df_ratings_3['rating'] < 0.5, 'rating'] = median

ratings2_nogen = df_ratings_3.loc[df_ratings_3['movieId'].isin(no_genres.movieId.tolist())]
ratings2_action = df_ratings_3.loc[df_ratings_3['movieId'].isin(action.movieId.tolist())]
ratings2_aventures = df_ratings_3.loc[df_ratings_3['movieId'].isin(aventures.movieId.tolist())]
ratings2_animation = df_ratings_3.loc[df_ratings_3['movieId'].isin(animation.movieId.tolist())]
ratings2_children = df_ratings_3.loc[df_ratings_3['movieId'].isin(children.movieId.tolist())]
ratings2_comedy = df_ratings_3.loc[df_ratings_3['movieId'].isin(comedy.movieId.tolist())]
ratings2_crime = df_ratings_3.loc[df_ratings_3['movieId'].isin(crime.movieId.tolist())]
ratings2_documentary = df_ratings_3.loc[df_ratings_3['movieId'].isin(documentary.movieId.tolist())]
ratings2_drama = df_ratings_3.loc[df_ratings_3['movieId'].isin(drama.movieId.tolist())]
ratings2_espionage = df_ratings_3.loc[df_ratings_3['movieId'].isin(espionage.movieId.tolist())]
ratings2_fantasy = df_ratings_3.loc[df_ratings_3['movieId'].isin(fantasy.movieId.tolist())]
ratings2_noir = df_ratings_3.loc[df_ratings_3['movieId'].isin(noir.movieId.tolist())]
ratings2_horror = df_ratings_3.loc[df_ratings_3['movieId'].isin(horror.movieId.tolist())]
ratings2_imax = df_ratings_3.loc[df_ratings_3['movieId'].isin(imax.movieId.tolist())]
ratings2_musical = df_ratings_3.loc[df_ratings_3['movieId'].isin(musical.movieId.tolist())]
ratings2_mystery = df_ratings_3.loc[df_ratings_3['movieId'].isin(mystery.movieId.tolist())]
ratings2_psycho = df_ratings_3.loc[df_ratings_3['movieId'].isin(thriller.movieId.tolist())]
ratings2_romance = df_ratings_3.loc[df_ratings_3['movieId'].isin(romance.movieId.tolist())]
ratings2_sci_fi = df_ratings_3.loc[df_ratings_3['movieId'].isin(sci_fi.movieId.tolist())]
ratings2_thriller = df_ratings_3.loc[df_ratings_3['movieId'].isin(thriller.movieId.tolist())]
ratings2_war = df_ratings_3.loc[df_ratings_3['movieId'].isin(war.movieId.tolist())]
ratings2_western = df_ratings_3.loc[df_ratings_3['movieId'].isin(western.movieId.tolist())]

ratings2 = list()
ratings2.append(ratings2_nogen['rating'].mean())
ratings2.append(ratings2_action['rating'].mean())
ratings2.append(ratings2_aventures['rating'].mean())
ratings2.append(ratings2_animation ['rating'].mean())
ratings2.append(ratings2_children ['rating'].mean())
ratings2.append(ratings2_comedy['rating'].mean())
ratings2.append(ratings2_crime['rating'].mean())
ratings2.append(ratings2_documentary['rating'].mean())
ratings2.append(ratings2_drama['rating'].mean())
ratings2.append(ratings2_espionage['rating'].mean())
ratings2.append(ratings2_fantasy['rating'].mean())
ratings2.append(ratings2_noir['rating'].mean())
ratings2.append(ratings2_horror['rating'].mean())
ratings2.append(ratings2_imax['rating'].mean())
ratings2.append(ratings2_musical ['rating'].mean())
ratings2.append(ratings2_mystery ['rating'].mean())
ratings2.append(ratings2_psycho['rating'].mean())
ratings2.append(ratings2_romance ['rating'].mean())
ratings2.append(ratings2_sci_fi['rating'].mean())
ratings2.append(ratings2_thriller['rating'].mean())
ratings2.append(ratings2_war['rating'].mean())
ratings2.append(ratings2_western['rating'].mean())

table_rat2 = pd.DataFrame(list(zip(list(genres)[1:],ratings2)), columns=['genre', 'rating'])

table_rat2.sort_values(by=['rating'], ascending=False).head(10)

C:\ProgramData\Miniconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\ProgramData\Miniconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,genre,rating
11,(no genres listed),3.924775
20,Animation,3.790411
7,Musical,3.704676
6,Psychological Thriller,3.684254
8,Drama,3.676340
15,Mystery,3.669411
3,Sci-Fi,3.614207
13,Espionage Action,3.602726
14,Fantasy,3.553766
17,Film-Noir,3.541947


Observamos como el remplazamiento de los valores de los outliers tiene un gran impacto, sobretodo porque el top 4 del ranking anterior ni siquiera aparece en el top 10 de este nuevo ranking. COncluimos que el método de remplazamiento de outliers y missing values puede llegar a ser crítico para sacar conclusiones, ya que diferentes métodos, nos pueden llegar a dar conclusiones distintas. Sin embargo, en este caso, mantenemos el método original, por considerar más razonable los valores dados a los outliers en el mismo, dada la naturaleza de los datos y el contexto de los mismos.